In [1]:
import pandas as pd
import requests
import time
import glob
import os

**Flight Data Aggregation and Date Range Splitting for Weather Data Fetching**

In [ ]:
"""
  After running this code, we will have a CSV file (`split_df.csv`) with city names, date ranges,
  and flags to indicate which date ranges still need weather data. This will help us fetch
  weather data for specific cities and date ranges.
"""

flight_df = pd.read_csv('./data/flights_sample_3m.csv')

a_df = flight_df.groupby(['ORIGIN_CITY']).agg({
    'FL_DATE': ['nunique', 'min', 'max']  # Get the unique count, min, and max of flight dates
}).reset_index()
a_df.columns = ['origin_city', 'nunique', 'fl_dt_min', 'fl_dt_max']

# Extract city name and city code from 'origin_city'
a_df['city_name'] = a_df['origin_city'].map(
    lambda x: x.split(',')[0].split('/')[0]  # Extract the city name from 'origin_city'
)

a_df['city_code'] = a_df['origin_city'].map(
    lambda x: x.split(',')[1]  # Extract the city code from 'origin_city'
)

a_df["fl_dt_min"] = a_df["fl_dt_min"].astype(str)
a_df["fl_dt_max"] = a_df["fl_dt_max"].astype(str)

a_df['fl_dt_min'] = pd.to_datetime(a_df['fl_dt_min'])
a_df['fl_dt_max'] = pd.to_datetime(a_df['fl_dt_max'])
a_df['day_diff'] = (a_df['fl_dt_max'] - a_df['fl_dt_min']).dt.days

# Split the date ranges into smaller chunks if the range exceeds a threshold
split_rows = []
THRESHOLD = 400  # The maximum number of days per chunk

for idx, row in a_df.iterrows():
    remaining_days = row['day_diff']
    current_start = row['fl_dt_min']

    while remaining_days > THRESHOLD:
        split_rows.append({
            'origin_city': row['origin_city'],
            'city_name': row['city_name'],
            'fl_dt_min': current_start,
            'fl_dt_max': current_start + pd.Timedelta(days=THRESHOLD),
            'day_diff': THRESHOLD
        })
        remaining_days -= (THRESHOLD + 1)

    # Add the final chunk if any remaining days are less than the threshold
    if remaining_days > 0:
        split_rows.append({
            'origin_city': row['origin_city'],
            'city_name': row['city_name'],
            'fl_dt_min': current_start,
            'fl_dt_max': row['fl_dt_max'],
            'day_diff': (row['fl_dt_max'] - current_start).days
        })

split_df = pd.DataFrame(split_rows)

split_df['flg_fetch'] = 0  # Flag to indicate if weather data has been fetched
split_df['start_date'] = split_df['fl_dt_min']  # Start date of the chunk
split_df['end_date'] = split_df['fl_dt_max']  # End date of the chunk

split_df.to_csv('./split_df.csv', index=False)

**Weather Data Fetching Script using Visual Crossing API**

In [ ]:
"""
    This script fetches weather data for cities and date ranges specified in the split_df file.
    It avoids fetching duplicate data by ensuring the 'flg_fetch' column is 0 (indicating weather data has not been fetched yet).
    Weather data is retrieved using the Visual Crossing API and stored as CSV files for each city and date range.
    After fetching, the 'flg_fetch' column is updated to 1 to indicate that the weather data for that specific period has been fetched.
"""

import pandas as pd
import requests
import time


split_df = pd.read_csv('./split_df.csv')

# Filter out the rows that haven't had weather data fetched yet (flg_fetch == 0)
split_df = split_df[split_df['flg_fetch'] == 0]


def get_weather_bulk(city, start_date, end_date, api_key):
    BASE_URL = "https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/"
    url = f"{BASE_URL}{city}/{start_date}/{end_date}?key={api_key}&include=days&contentType=json"

    response = requests.get(url)
    if response.status_code == 200:
        data = response.json().get("days", [])
        if data:
            df = pd.DataFrame(data)
            df['city'] = city
            df['start_date'] = start_date
            df['end_date'] = end_date
            return df
        else:
            print(f"No data available for {city} between {start_date} and {end_date}.")
            return pd.DataFrame()
    else:
        print(f"Error fetching data for {city}: HTTP {response.status_code} - {response.text}")
        return pd.DataFrame()

# List of API keys for authentication with the Visual Crossing API
api_keys =  ['ELDKA2BG6NKRWNDM5776PVRPL', 'EYT2VCQPMAXV6ZK77MWRCZBN7', 'ZN4Y72MP3G5S98RQZ8M9LN2BE',
             'AVEMEQN8ZL45N27TQF7RQDJGQ', '5CHTDUCBRZBKR5ZFL3HC2VB75', 'U6WVN4A72JUEEW3N7SLZXM7CN',
             '7FVJZKNA9J7B4RV4VSDF3YLM7', 'E7JRB82RB5LND8SDG6NCR4LEB', '68G8M3YHM5E29DA7C2Y7AFKUY',
             'P427H269UM2TERXN6TK2XKMAG', 'QFP292BSQWU5KNCQWFZGCPSCQ', 'JLK2CTBFUGFYVRD4L7LW3B3K9',
             'VDMDB4UW3M8QXNFWE7W2JEMGT', 'ABD6CU8YX59M77DMUPV6BURKB', 'WJHYBMW5GKBPHHX23E5MPTJR9',
             'KVA5FXXK97LNPM9J7JXHR8KXL', 'PYLL32BJ6GQEY7E3CQDEWSB9V', '5JLE7QCYC28GARPDETXWZ67ZV',
             'WTFP829GRTQC3XXJYTBFMRHBK', 'F8D3NZPVTVUEJ9RHPAGRU5KP5', '4EXDMKUJYAKUAPD9ZBE99CG8P',
             'L767TUTXJ7N4AAF4CF3PM6PFW', '3QCMGWNLNPUVDUFGTASLMPEFL', 'LZWNLC9PKC2VCE5KBG9MKDMR2',
             'P9EAPE7W2VHGRQRZ4248F4KJC']


i = 0
dummy_df = pd.DataFrame()

while i < len(api_keys):
    print(f"Using API key: {api_keys[i]}")
    split_df = split_df[split_df['flg_fetch'] == 0]

    for idx, row in split_df.iterrows():
        print(f"Fetching weather data for {row['city_name']} from {row['start_date']} to {row['end_date']}")
        df = get_weather_bulk(row['city_name'], row['start_date'], row['end_date'], api_keys[i])


        time.sleep(1)

        if not df.empty:
            df.to_csv(f'./weather_data/{row["city_name"]}_{row["start_date"]}_{row["end_date"]}.csv')
            split_df.at[idx, 'flg_fetch'] = 1
        else:
            print(f"Received empty data for {row['city_name']} between {row['start_date']} and {row['end_date']}.")
            i += 1
            break


split_df.to_csv('./split_df.csv', index=False)

"""
At this point, the weather data has been fetched and saved for the specified cities and date ranges,
and the split_df file has been updated to reflect the progress.
"""

**Merging Weather Data Files into a Single CSV for Analysis**

In [ ]:
"""
Now that we have gathered all the weather data in individual files, we will combine them into a single file for analysis purposes.
"""
folder_path = './weather_data/'

csv_files = glob.glob(os.path.join(folder_path, '*.csv'))

dfs = []
i = 0
for file in csv_files:
    df = pd.read_csv(file)
    df['SourceFile'] = os.path.basename(file)  # Add a column to track the source file
    dfs.append(df)
    i += 1
    print(f"Processing file {i}: {file}")


combined_df = pd.concat(dfs, ignore_index=True)


combined_df.to_csv('/content/drive/MyDrive/data_mining/project/data/combined_weather_data.csv')
